In [4]:
!pip install -q -r requirements.txt

ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'


In [5]:
import json
import requests
import pandas as pd
import pickle
from datetime import datetime, timezone

data = pd.read_json("data.json")
data_supp = pd.read_json("data_supp.json")
data_distr = pd.read_json("data_distr.json")
data_main = pd.read_json("data_main.json")

In [6]:
data_cols = ["AanestysId","TekninenAvain","IstuntoTyyppi","IstuntoTila","IstuntoTilaSeliteFI","IstuntoTilaSeliteSV","IstuntoVPVuosi","IstuntoNumero","IstuntoPvm","IstuntoIlmoitettuAlkuaika","IstuntoAlkuaika","IstuntoLoppuaika","IstuntoNimenhuutoaika","KasiteltavaKohtaTekninenAvain","PJTekninenAvain","PJOtsikkoFI","PJOtsikkoSV","PJTila","XmlData","Created","Modified","PuhujaHenkilonumero","ManuaalinenEsto","AttachmentGroupId","Imported"]
data_supp_cols = ["AanestysId","KieliId","IstuntoVPVuosi","IstuntoNumero","IstuntoPvm","IstuntoIlmoitettuAlkuaika","IstuntoAlkuaika","PJOtsikko","AanestysNumero","AanestysAlkuaika","AanestysLoppuaika","AanestysMitatoity","AanestysOtsikko","AanestysLisaOtsikko","PaaKohtaTunniste","PaaKohtaOtsikko","PaaKohtaHuomautus","KohtaKasittelyOtsikko","KohtaKasittelyVaihe","KohtaJarjestys","KohtaTunniste","KohtaOtsikko","KohtaHuomautus","AanestysTulosJaa","AanestysTulosEi","AanestysTulosTyhjia","AanestysTulosPoissa","AanestysTulosYhteensa","Url","AanestysPoytakirja","AanestysPoytakirjaUrl","AanestysValtiopaivaasia","AanestysValtiopaivaasiaUrl","AliKohtaTunniste","Imported"]
data_distr_cols = ["JakaumaId","AanestysId","Ryhma","Jaa","Ei","Tyhjia","Poissa","Yhteensa","Tyyppi","Imported"]
data_main_cols = ["AanestysId","KieliId","IstuntoVPVuosi","IstuntoNumero","IstuntoPvm","IstuntoIlmoitettuAlkuaika","IstuntoAlkuaika","PJOtsikko","AanestysNumero","AanestysAlkuaika","AanestysLoppuaika","AanestysMitatoity","AanestysOtsikko","AanestysLisaOtsikko","PaaKohtaTunniste","PaaKohtaOtsikko","PaaKohtaHuomautus","KohtaKasittelyOtsikko","KohtaKasittelyVaihe","KohtaJarjestys","KohtaTunniste","KohtaOtsikko","KohtaHuomautus","AanestysTulosJaa","AanestysTulosEi","AanestysTulosTyhjia","AanestysTulosPoissa","AanestysTulosYhteensa","Url","AanestysPoytakirja","AanestysPoytakirjaUrl","AanestysValtiopaivaasia","AanestysValtiopaivaasiaUrl","AliKohtaTunniste","Imported"]

data.columns = data_cols
data_supp.columns = data_supp_cols
data_distr.columns = data_distr_cols
data_main.columns = data_main_cols

In [7]:

print("convert some values to numerical type")

data["AanestysId"] = data["AanestysId"].apply(pd.to_numeric, errors="coerce")
data_supp["AanestysId"] = data_supp["AanestysId"].apply(pd.to_numeric, errors="coerce")
data_distr["AanestysId"] = data_distr["AanestysId"].apply(pd.to_numeric, errors="coerce")
data_distr["Jaa"] = data_distr["Jaa"].apply(pd.to_numeric, errors="coerce")
data_distr["Ei"] = data_distr["Ei"].apply(pd.to_numeric, errors="coerce")
data_main["AanestysId"] = data_main["AanestysId"].apply(pd.to_numeric, errors="coerce")
data_main["AanestysTulosJaa"] = data_main["AanestysTulosJaa"].apply(pd.to_numeric, errors="coerce")
data_main["AanestysTulosEi"] = data_main["AanestysTulosEi"].apply(pd.to_numeric, errors="coerce")
data_main["IstuntoVPVuosi"] = data_main["IstuntoVPVuosi"].apply(pd.to_numeric, errors="coerce")

convert some values to numerical type


In [ ]:


print("fix data with empty columns")

data.dropna()
data_supp.dropna()
data_distr.dropna()
data_main.dropna()

#merged = pd.merge(data_supp, data_distr, on="AanestysId")
# print("data main:")
# print(data_main.info()) 


In [1]:

voted = data_main.loc[
    (data_main["IstuntoVPVuosi"] > 2014)
]

print(len(voted))

voted = voted.loc[
    (((voted["AanestysTulosJaa"] > 40) &
    (voted["AanestysTulosEi"] > 25)) &

    ((voted['KohtaOtsikko'].str.contains('hiil', case=False) == True) |
    (voted['KohtaOtsikko'].str.contains('energiamuoto', case=False) == True) |
    (voted['KohtaOtsikko'].str.contains('uusiutuv', case=False) == True) |
    (voted['KohtaOtsikko'].str.contains('sähköverk', case=False) == True) |
    (voted['KohtaOtsikko'].str.contains('energians', case=False) == True) |
    (voted['KohtaOtsikko'].str.contains('tartuntatauti', case=False) == True) |
    (voted['KohtaOtsikko'].str.contains('hoiva', case=False) == True) |
    (voted['KohtaOtsikko'].str.contains('hoit', case=False) == True) |
    (voted['KohtaOtsikko'].str.contains('koulutu', case=False) == True) |
    (voted['KohtaOtsikko'].str.contains('kasviruo', case=False) == True) |
    (voted['KohtaOtsikko'].str.contains('rahoitu', case=False) == True) |
    (voted['KohtaOtsikko'].str.contains('luonnonsuo', case=False) == True) |
    (voted['KohtaOtsikko'].str.contains('monimuot', case=False) == True) |
    (voted['KohtaOtsikko'].str.contains('euroop', case=False) == True) |
    (voted['KohtaOtsikko'].str.contains('unioni', case=False) == True) |
    (voted['KohtaOtsikko'].str.contains('yle', case=False) == True) |
    (voted['KohtaOtsikko'].str.contains('eläke', case=False) == True))) |
    
    ((voted['KohtaOtsikko'].str.contains('työtaistel', case=False) == True) |
    (voted['KohtaOtsikko'].str.contains('saimaan', case=False) == True) |
    (voted['KohtaOtsikko'].str.contains('raska', case=False) == True) |
    (voted['KohtaOtsikko'].str.contains('uusiutuv', case=False) == True) |
    (voted['KohtaOtsikko'].str.contains('unioni', case=False) == True) |
    (voted['KohtaOtsikko'].str.contains('eu:', case=False) == True) |
    (voted['KohtaOtsikko'].str.contains('ilmast', case=False) == True) |
    (voted['KohtaOtsikko'].str.contains('trans', case=False) == True) |
    (voted['KohtaOtsikko'].str.contains('sukupuol', case=False) == True) &
    
    ((voted["AanestysTulosJaa"] > 20) |
    (voted["AanestysTulosEi"] > 15))
    )
    ]


voted = voted[voted.apply(lambda row: row.astype(str).str.contains('hyv', case=False).any(), axis=1)]
voted = voted[voted.apply(lambda row: row.astype(str).str.contains('hyl', case=False).any(), axis=1)]


#voted = voted.loc[
#    (voted['AanestysLisaOtsikko'].str.contains('hyv', case=False) == True) |
#    (voted['AanestysLisaOtsikko'].str.contains('hylk', case=False) == True) | 
#    (voted['AanestysOtsikko'].str.contains('hyv', case=False) == True) | 
#    (voted['AanestysOtsikko'].str.contains('hylk', case=False) == True) ]

print(len(voted))

for i, row in voted.iterrows():
    print(row['KohtaOtsikko'], row["AanestysPoytakirjaUrl"], row['AanestysId'])

voted_distr = data_distr[data_distr['AanestysId'].isin(list(voted["AanestysId"]))]
voted_distr = voted_distr[voted_distr['Tyyppi'].str.contains("eduskuntaryhma", case=False)]

print(len(voted_distr))

voted_distr.to_csv('aanestykset_distr2.csv', encoding='utf-8')
voted.to_csv('aanestykset2.csv', encoding='utf-8')


# for i, row in voted_distr.iterrows():
#     print(row)

# turvallisuuslaki = data_supp.loc[data_supp['KohtaOtsikko'].str.contains("potilasturvallisuu", case=False, na=False)]

# for item in turvallisuuslaki["KohtaOtsikko"]:
#     print(item)
